<a href="https://colab.research.google.com/github/mcarhuasn/DIA2022-VisionArtif-ClasificacionCOVID/blob/master/Proyecto_Clasificacion_COVID19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<h1><font color="#113D68" size=5.5>Proyecto: Clasificación de Pacientes con COVID-9 usando imágenes de Rayos X</font></h1>**

<h1><font color="#113D68" size=5>Curso: Desarrollo de Aplicaciones con Visión Artificial</font></h1>

<h1><font color="#113D68" size=4>Docentes: Dr. Analí Alfaro Alfaro, Mg. Alaín Alejo Huarachi</font></h1>

**<h1><font color="#113D68" size=4>Grupo**:

- Milton Carhuas Ñañez

- Carlos Incasoller Vilca

</font></h1>

<a id="section23"></a>
## <font color="#004D7F"> Importación de Librerías</font>

In [13]:
import torchvision
from PIL import Image
import torchvision.transforms as T
import cv2
import plotly.express as px
import os, sys, time, datetime, random

import os
import shutil

import pandas as pd
import numpy as np

# importando librerías
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
import plotly.express as px
import time
import copy

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. RECUPERACIÓN DE INFORMACIÓN</font>

#### Creación de variables para directorios y archivos

In [38]:
# Directorios
DIR_PROY = '/content/drive/MyDrive/Diplomado-Dev-Aplic-IA/05_Des_Aplic_VisionArtificial/Proyecto_VA'
DIR_REPOS_PROY = DIR_PROY + '/covid-chestxray-dataset'
DIR_METADATA = DIR_REPOS_PROY
DIR_DATA = DIR_REPOS_PROY + '/images'
DIR_COVID = DIR_DATA + "/covid"
DIR_NO_COVID = DIR_DATA + "/no-covid"

# Archivos
FILE_METADATA = 'metadata.csv'

#### Habilitar acceso a directorios de Google Drive

In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
!pwd

/content


In [3]:
!ls -ltrh /content/

total 8.0K
drwxr-xr-x 1 root root 4.0K Jun 15 13:42 sample_data
drwx------ 6 root root 4.0K Jun 29 17:11 drive


In [4]:
%cd "/content/drive/MyDrive/Diplomado-Dev-Aplic-IA/05_Des_Aplic_VisionArtificial/Proyecto_VA/"

/content/drive/MyDrive/Diplomado-Dev-Aplic-IA/05_Des_Aplic_VisionArtificial/Proyecto_VA


In [6]:
! git clone https://github.com/ieee8023/covid-chestxray-dataset

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 3641, done.
remote: Total 3641 (delta 0), reused 0 (delta 0), pack-reused 3641
Receiving objects: 100% (3641/3641), 632.96 MiB | 17.62 MiB/s, done.
Resolving deltas: 100% (1450/1450), done.
Checking out files: 100% (1174/1174), done.


In [8]:
# Tamaño del directorio de imagenes
!du -h "/content/drive/MyDrive/Diplomado-Dev-Aplic-IA/05_Des_Aplic_VisionArtificial/Proyecto_VA/covid-chestxray-dataset/images"

527M	/content/drive/MyDrive/Diplomado-Dev-Aplic-IA/05_Des_Aplic_VisionArtificial/Proyecto_VA/covid-chestxray-dataset/images


In [9]:
# Cantidad de elementos del directorio de imagenes
!ls "/content/drive/MyDrive/Diplomado-Dev-Aplic-IA/05_Des_Aplic_VisionArtificial/Proyecto_VA/covid-chestxray-dataset/images" | wc -l

930


In [10]:
# Tipos de archivo del directorio de imagenes
!find "/content/drive/MyDrive/Diplomado-Dev-Aplic-IA/05_Des_Aplic_VisionArtificial/Proyecto_VA/covid-chestxray-dataset/images" -type f | awk -F. '!a[$NF]++{print $NF}'

jpg
png
jpeg
PNG
JPG


#### Importación de Metadata (Etiqueta por cada Imagen)

In [14]:
df_metadata = pd.read_csv(DIR_METADATA + FILE_METADATA, sep=',')
print('Tamaño del conjunto de datos: ', df_metadata.shape)
df_metadata.head(5)

Tamaño del conjunto de datos:  (950, 30)


,patientid,offset,sex,age,finding,RT_PCR_positive,survival,intubated,intubation_present,went_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 29
0,2,0.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
1,2,3.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
2,2,5.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
3,2,6.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
4,4,0.0,F,52.0,Pneumonia/Viral/COVID-19,Y,NaN,N,N,N,...,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN


In [15]:
# Cantidad de imágenes para enfermedad del tipo COVID-9
df_metadata[df_metadata["finding"]=='Pneumonia/Viral/COVID-19'].shape

(584, 30)

In [16]:
# Cantidad de imágenes por tipo de enfermedad
pd.DataFrame(df_metadata["finding"].value_counts(dropna=False))

,finding
Pneumonia/Viral/COVID-19,584
todo,83
Pneumonia,81
Pneumonia/Fungal/Pneumocystis,30
Pneumonia/Bacterial/Streptococcus,22
No Finding,22
Tuberculosis,18
Pneumonia/Viral/SARS,16
Pneumonia/Lipoid,13
Pneumonia/Bacterial/Mycoplasma,11


In [17]:
# Cantidad de imágenes por tipo de carpeta
pd.DataFrame(df_metadata["folder"].value_counts(dropna=False))

,folder
images,929
volumes,21


#### Creación de Variable Objetivo

In [19]:
df_metadata['label'] = [1 if x == 'Pneumonia/Viral/COVID-19' else 0 for x in df_metadata['finding']]
print(df_metadata.shape)
df_metadata[['finding','folder','filename','label']].head(5)

(950, 31)


,finding,folder,filename,label
0,Pneumonia/Viral/COVID-19,images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,1
1,Pneumonia/Viral/COVID-19,images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,1
2,Pneumonia/Viral/COVID-19,images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,1
3,Pneumonia/Viral/COVID-19,images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,1
4,Pneumonia/Viral/COVID-19,images,nejmc2001573_f1a.jpeg,1


#### Definición de Universo

In [21]:
pd.set_option('display.max_colwidth',-1)

df_metadata_u = df_metadata.copy()
df_metadata_u = df_metadata_u[['finding','folder','filename','label']][df_metadata["folder"]=='images']
print('Tamaño del conjunto de datos: ', df_metadata_u.shape)
df_metadata_u.head(5)

Tamaño del conjunto de datos:  (929, 4)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,finding,folder,filename,label
0,Pneumonia/Viral/COVID-19,images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg,1
1,Pneumonia/Viral/COVID-19,images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg,1
2,Pneumonia/Viral/COVID-19,images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg,1
3,Pneumonia/Viral/COVID-19,images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg,1
4,Pneumonia/Viral/COVID-19,images,nejmc2001573_f1a.jpeg,1


In [22]:
# Cantidad de imágenes por tipo de enfermedad para Universo
pd.DataFrame(df_metadata_u["finding"].value_counts(dropna=False))

,finding
Pneumonia/Viral/COVID-19,563
todo,83
Pneumonia,81
Pneumonia/Fungal/Pneumocystis,30
Pneumonia/Bacterial/Streptococcus,22
No Finding,22
Tuberculosis,18
Pneumonia/Viral/SARS,16
Pneumonia/Lipoid,13
Pneumonia/Bacterial/Mycoplasma,11


#### Clasificación de imágenes por clase (COVID / NO-COVID)

In [31]:
# DIR_COVID = DIR_DATA + "covid/"
# DIR_NO_COVID = DIR_DATA + "no-covid/"

os.mkdir(DIR_COVID)
print("Directory '% s' created" % DIR_COVID)

os.mkdir(DIR_NO_COVID)
print("Directory '% s' created" % DIR_NO_COVID)

Directory '/content/drive/MyDrive/Diplomado-Dev-Aplic-IA/05_Des_Aplic_VisionArtificial/Proyecto_VA/covid-chestxray-dataset/images/covid/' created
Directory '/content/drive/MyDrive/Diplomado-Dev-Aplic-IA/05_Des_Aplic_VisionArtificial/Proyecto_VA/covid-chestxray-dataset/images/no-covid/' created


In [ ]:
count = 0

for path, dirs, files in os.walk(DIR_DATA):
  for filename in files:
    #print(os.path.join(filename))
    #print(filename)
    dir_orig = os.path.join(DIR_DATA, filename)

    for idx, row in df_metadata_u.iterrows():
      if row['filename'] == filename and row['label'] == 1:
        count = count + 1
        #print('Indice DF: ', idx, ' | Imagen COVID: ', row['filename'], ' $ ' , filename, ' | Contador: ', count)

        shutil.move(dir_orig, DIR_COVID)
      elif row['filename'] == filename and row['label'] != 1:
        shutil.move(dir_orig, DIR_NO_COVID)

print('Cantidad de imagenes COVID movidas a nueva carpeta: ', count)

In [46]:
# Cantidad de archivos en carpeta "covid"
!ls "/content/drive/MyDrive/Diplomado-Dev-Aplic-IA/05_Des_Aplic_VisionArtificial/Proyecto_VA/covid-chestxray-dataset/images/covid" | wc -l

563


In [47]:
# Cantidad de archivos en carpeta "covid"
!ls "/content/drive/MyDrive/Diplomado-Dev-Aplic-IA/05_Des_Aplic_VisionArtificial/Proyecto_VA/covid-chestxray-dataset/images/no-covid" | wc -l

366


---
<a id="section0"></a>
# <font color="#004D7F" size=6> 1. ENTENDIMIENTO DEL CONTEXTO / NEGOCIO</font>

Colección de dataset de imágenes de COVID-9:

https://github.com/ieee8023/covid-chestxray-dataset

---
<a id="section0"></a>
# <font color="#004D7F" size=6>2. ENTENDIMIENTO DE LOS DATOS</font>

---
<a id="section0"></a>
# <font color="#004D7F" size=6>3. PREPARACIÓN DE LOS DATOS</font>

<a id="section0"></a>
# <font color="#004D7F" size=6>4. MODELADO DE LOS DATOS</font>

<a id="section0"></a>
# <font color="#004D7F" size=6>5. EVALUACIÓN</font>